In [1]:
import pandas, urllib, datetime

import pandas.io.sql as psql
from sqlalchemy import create_engine

params = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER=%s,%s;DATABASE=%s;UID=%s;PWD=%s'%('localhost', '1433', 'ThuctapSNP', 'sa', 'Matkhau1234'))
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s'%params)

In [2]:
ITEM = pandas.read_sql('select * from ITEM', engine)
ITEM_REEFER = pandas.read_sql('select * from ITEM_REEFER', engine)
ITEM_DANGEROUS = pandas.read_sql('select * from ITEM_DANGEROUS', engine)
ITEM_OOG = pandas.read_sql('select * from ITEM_OOG', engine)
ITEM_LOCATION = pandas.read_sql('select * from ITEM_LOCATION', engine)
PREGATE_TRANSACT = pandas.read_sql('select * from PREGATE_TRANSACT', engine)
YARD_AREA = pandas.read_sql('select * from YARD_AREA', engine)

In [3]:
item = ITEM.loc[
    (ITEM['SITE_ID'].str.strip()=='CTL') & 
    (ITEM['ARR_BY'].str.strip()=='T') & 
    (ITEM['HIST_FLG'].str.strip()=='Y') & 
    (ITEM['FEL'].str.strip()=='F')]
item = item.drop(['ITEM_TYPE'], axis=1)

In [7]:
ITEM_REEFER['ITEM_TYPE'] = 'R'
reefer = pandas.merge(ITEM_REEFER, item, how='inner', on='ITEM_KEY')
reefer = reefer[['ITEM_KEY', 'ITEM_TYPE',]]

ITEM_DANGEROUS['ITEM_TYPE'] = 'D'
dangerous = pandas.merge(ITEM_DANGEROUS, item, how='inner', on='ITEM_KEY')
dangerous = dangerous[['ITEM_KEY', 'ITEM_TYPE']]

ITEM_OOG['ITEM_TYPE'] = 'O'
oog = pandas.merge(ITEM_OOG, item, how='inner', on='ITEM_KEY')
oog = oog[['ITEM_KEY', 'ITEM_TYPE']]

In [8]:
item_special = pandas.concat([reefer, dangerous, oog], axis=0)

In [9]:
item = item[['ITEM_KEY', 'ITEM_NO', 'LENGTH', 'ARR_TS', 'DEP_TS']]

In [10]:
item = pandas.merge(item, item_special, how='left', on=['ITEM_KEY']).fillna({'ITEM_TYPE':'G'})

In [46]:
unique_location = set([tuple(row) for i, row in YARD_AREA[['REGION', 'AREA', 'STACK']].iterrows()])
while len(unique_location) > 0:
    engine.execute('insert into d_LOCATION values {}'.format(unique_location.pop()))

In [11]:
location = pandas.read_sql('select * from d_LOCATION', engine)

In [12]:
pregate = PREGATE_TRANSACT[['ITEM_KEY', 'R_D', 'AREA']]
item_loc = ITEM_LOCATION[['ITEM_KEY', 'STACK', 'EXEC_TS']]

In [13]:
stg_transact = pandas.merge(item, pregate, how='inner', on='ITEM_KEY').dropna(subset=['R_D'])
stg_transact = stg_transact.reset_index(drop=True)

In [31]:
stg_transact['EXEC_TS'] = None
stg_transact['STACK'] = None
for i in range(len(stg_transact)):
    row = stg_transact.loc[i]
    item_key = row['ITEM_KEY']
    r_d = row['R_D'].strip()
    if r_d == 'R':
        if row['ARR_TS'].to_pydatetime().year > 1900:
            stg_transact.loc[i, 'EXEC_TS'] = row['ARR_TS']
            try:
                stack = item_loc.loc[(item_loc['ITEM_KEY']==item_key) & (item_loc['EXEC_TS']>row['ARR_TS'])].sort_values(['EXEC_TS'], ascending=[True])['STACK'].tolist()[0]
                stg_transact.loc[i, 'STACK'] = stack
            except:
                pass
        else:
            stg_transact.drop(i)
    elif r_d == 'D':
        if row['DEP_TS'].to_pydatetime().year > 1900:
            stg_transact.loc[i, 'EXEC_TS'] = row['DEP_TS']
            try:
                stack = item_loc.loc[(item_loc['ITEM_KEY']==item_key) & (item_loc['EXEC_TS']>row['DEP_TS'])].sort_values(['EXEC_TS'], ascending=[False])['STACK'].tolist()[0]
                stg_transact.loc[i, 'STACK'] = stack
            except:
                pass
        else:
            stg_transact.drop(i)

In [62]:
sample = pandas.merge(stg_transact, item_loc, how='left', on='ITEM_KEY')

In [22]:
stg_transact['DURATION'] = stg_transact['DEP_TS'] - stg_transact['ARR_TS']

In [23]:
stg_transact['DURATION'] = stg_transact['DURATION'].apply(pandas.Timedelta.total_seconds)

In [41]:
stg_transact

,ITEM_KEY,ITEM_NO,LENGTH,ARR_TS,DEP_TS,ITEM_TYPE,R_D,AREA,EXEC_TS,STACK,DURATION
0,12767774,YMLU3451130,20.0,2019-01-01 00:00:10,2019-01-07 18:44:54,G,R,KHX,2019-01-01 00:00:10,K09,585884.0
1,12767781,MNBU3459229,40.0,2019-01-01 00:01:03,2019-01-03 01:27:53,R,R,KHL,2019-01-01 00:01:03,L32,178010.0
2,12767788,MNBU0228951,40.0,2019-01-01 00:03:41,2019-01-03 01:23:05,R,R,KHL,2019-01-01 00:03:41,L32,177564.0
3,12767793,KKFU6769433,40.0,2019-01-01 00:05:07,2019-01-04 02:48:36,R,R,KHL,2019-01-01 00:05:07,L28,269009.0
4,12767793,KKFU6769433,40.0,2019-01-01 00:05:07,2019-01-04 02:48:36,R,D,KHL,2019-01-04 02:48:36,None,269009.0
...,...,...,...,...,...,...,...,...,...,...,...
17825,20078545,WHSU2090123,20.0,2020-11-01 00:29:30,2020-11-04 08:24:40,G,R,GNL,2020-11-01 00:29:30,Q04,287710.0
17826,20078544,DFSU2179820,20.0,2020-11-01 00:29:34,2020-11-03 00:10:56,G,D,None,2020-11-03 00:10:56,None,171682.0
17827,20078544,DFSU2179820,20.0,2020-11-01 00:29:34,2020-11-03 00:10:56,G,R,KHN,2020-11-01 00:29:34,D14,171682.0
17828,20078548,BEAU2527641,20.0,2020-11-01 00:29:42,2020-11-08 08:22:29,G,R,GNL,2020-11-01 00:29:42,Q04,633167.0


In [46]:
stg_transact = stg_transact.astype({
    'LENGTH' : 'float',
    'EXEC_TS' : 'datetime64',
    'DURATION' : 'int64'
})

In [51]:
stg_transact = stg_transact[['ITEM_KEY', 'ITEM_NO', 'LENGTH', 'AREA', 'STACK', 'R_D', 'DURATION', 'EXEC_TS', 'ITEM_TYPE']]

In [52]:
import pandas.io.sql as psql

In [55]:
psql.to_sql(stg_transact, 'stg_TRANSACT', engine, if_exists='append', index=False)

-1